# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42, **kwargs):
      super(Network, self).__init__()
      self.seed = torch.manual_seed(seed)
      # first variable
      self.fc1 = nn.Linear(state_size, 64)
      self.fc2 = nn.Linear(64, 64)
      self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    # update the value of x and call the functional module
    # relu = rectifier function
    x = F.relu(x)

    x = self.fc2(x)
    x = F.relu(x)

    # return the signal in the output layer
    return self.fc3(x)


## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
# create a new variable for the environment
env = gym.make('LunarLander-v2')
# get parameters for training: state shape, size, number of actions, etc
state_shape = env.observation_space.shape
# number of elements in the input state
state_size = env.observation_space.shape[0]
# we need to get the number of actions using action space attribute
number_actions = env.action_space.n
# printing to ensure we have the right parameters
print('State shape: ', state_shape) # vector of 8 elements
print('State size: ', state_size)
print('Number of actions: ', number_actions) # we have four


State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [6]:
class ReplayMemory(object):
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity # maximum size of the memory buffer
    self.memory = [ ]

  def push(self, event):
    self.memory.append(event) # we want to add the event into the memory list
    # make sure the memory buffer does not exceed its capacity
    if len(self.memory) > self.capacity:
      # we will remove the oldest event
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [7]:
class Agent():
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = Network(state_size, action_size).to(self.device)
        self.target_qnetwork = Network(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
        self.memory = ReplayMemory(replay_buffer_size)
        self.time_step = 0

    def step(self, state, action, reward, next_state, done):
        # Push experience into the replay memory
        self.memory.push((state, action, reward, next_state, done))
        # Increment the time step
        self.time_step = (self.time_step + 1) % 4

        # Every 4 steps, sample from memory and learn
        if self.time_step == 0:
            if len(self.memory.memory) > minibatch_size:
                experiences = self.memory.sample(minibatch_size)
                self.learn(experiences, discount_factor)

    def act(self, state, epsilon=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_qnetwork.eval()  # Set the network in evaluation mode
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()  # Set back to training mode

        # Epsilon-greedy action selection
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())  # Exploit
        else:
            return random.choice(np.arange(self.action_size))  # Explore

    def learn(self, experiences, discount_factor):
        # Unpack the experiences
        states, next_states, actions, rewards, dones = experiences

        # Get max predicted Q values for next states from target model
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)

        # Compute Q targets for current states
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))

        # Get expected Q values from local model
        q_expected = self.local_qnetwork(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(q_expected, q_targets)

        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_parameter):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [8]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [9]:
# initalize the training parameters
number_episodes = 2000
# maximum number of time steps per episode
maxNum_timesteps_per_episode = 1000
epsilon_start_val = 1.0
# we will let the epsilon value decay
# reduce it little by little to test other epsilon values
# epsilon ending value
epsilon_end_val = 0.01
epsilon_decay_val = 0.995
# create and initialize epsilon variable
epsilon = epsilon_start_val
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state = env.reset()[0]  # returns initial state (ensure it works for your env)
  # initalize the score, aka accumulative reward
  score = 0
  # loop over the time steps over the episode
  for t in range(maxNum_timesteps_per_episode):
    # select an action using act method: selects an action in a given state of env following epsilon-greedy policy
    action = agent.act(state, epsilon)
    # after making this move, it ends up in a new state of the environment
    # now we have the next state and reward
    next_state, reward, done, _, _ = env.step(action)
    # step method that includes the learn method
    agent.step(state, action, reward, next_state, done)
    # update the state variable because we are at the next state
    state = next_state
    # update the score
    score += reward
    # is our episode done?
    if done:
      break

  scores_on_100_episodes.append(score)
  # decay the epsilon while ensuring it does not go below 0.01
  epsilon = max(epsilon_end_val, epsilon_decay_val * epsilon)

  # advanced dynamic print to print dynamic scores
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")

  # make a new if condition that checks whether we are every 100 episodes
  if episode % 100 == 0:
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

  # an episode is considered a solution if it scores at least 200 points
  if np.mean(scores_on_100_episodes) >= 200.0:
      print('\nEnvironment Solved In: {:d} episodes\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
      # save null parameters
      torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
      # exit the training loop
      break

Episode 100	Average Score: -156.00
Episode 200	Average Score: -99.34
Episode 300	Average Score: -44.42
Episode 400	Average Score: -12.72
Episode 500	Average Score: 129.98
Episode 600	Average Score: 126.97
Episode 700	Average Score: 128.55
Episode 800	Average Score: 191.82
Episode 836	Average Score: 200.05
Environment Solved In: 736 episodes	Average Score: 200.05


## Part 3 - Visualizing the results

In [18]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()